In [8]:
import pandas as pd
import os
import html2text
import json

In [72]:
filename = "all_toots_mastodon_social.csv"
path_data = "./data/"

data = pd.read_csv(os.path.join(path_data, filename), sep=";")
data.head()

/var/folders/2h/923cq6912sqb0snfvqqfdnmm0000gn/T/ipykernel_91844/2295648728.py:4: DtypeWarning: Columns (0,2,3,4,10,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(os.path.join(path_data, filename), sep=";")


,toot_id,created_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,url,...,reblog,account,media_attachments,mentions,tags,emojis,card,poll,application,instance
0,110344178529920798,2023-05-10 11:35:41+00:00,NaN,NaN,False,NaN,public,it,https://botsin.space/users/sicetsimpliciter/st...,https://botsin.space/@sicetsimpliciter/1103441...,...,NaN,"{'id': 586424, 'username': 'sicetsimpliciter',...","[{'id': 110344178401470946, 'type': 'image', '...",[],[],[],NaN,NaN,NaN,mastodon.social
1,110344178520606732,2023-05-10 11:35:43+00:00,NaN,NaN,False,NaN,public,en,https://mastodon.me.uk/users/30yrdscreamer/sta...,https://mastodon.me.uk/@30yrdscreamer/11034417...,...,NaN,"{'id': 109291899427423346, 'username': '30yrds...",[],[],"[{'name': 'learnfromexperience', 'url': 'https...",[],NaN,NaN,NaN,mastodon.social
2,110344178517065175,2023-05-10 11:35:44+00:00,NaN,NaN,False,NaN,public,it,https://livellosegreto.it/users/chiagio/status...,https://livellosegreto.it/@chiagio/11034417849...,...,NaN,"{'id': 108072869854405865, 'username': 'chiagi...",[],[],"[{'name': 'discodelgiorno', 'url': 'https://ma...",[],NaN,NaN,NaN,mastodon.social
3,110344178514703217,2023-05-10 11:35:44.232000+00:00,NaN,NaN,False,NaN,public,en,https://mastodon.social/users/aitala/statuses/...,https://mastodon.social/@aitala/11034417851470...,...,NaN,"{'id': 108353362183208301, 'username': 'aitala...",[],[],[],[],NaN,NaN,"{'name': 'Web', 'website': None}",mastodon.social
4,110344178503475816,2023-05-10 11:35:40+00:00,NaN,NaN,False,NaN,public,en,https://mastodon.sdf.org/users/dkluft/statuses...,https://mastodon.sdf.org/@dkluft/1103441782781...,...,NaN,"{'id': 109293500780368127, 'username': 'dkluft...","[{'id': 110344178356278725, 'type': 'image', '...",[],"[{'name': 'legalethics', 'url': 'https://masto...",[],NaN,NaN,NaN,mastodon.social


In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135566 entries, 0 to 135565
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   toot_id                 135566 non-null  object 
 1   created_at              135560 non-null  object 
 2   in_reply_to_id          9427 non-null    object 
 3   in_reply_to_account_id  9427 non-null    object 
 4   sensitive               135560 non-null  object 
 5   spoiler_text            3017 non-null    object 
 6   visibility              135559 non-null  object 
 7   language                120510 non-null  object 
 8   uri                     135559 non-null  object 
 9   url                     135558 non-null  object 
 10  replies_count           135559 non-null  object 
 11  reblogs_count           135558 non-null  float64
 12  favourites_count        135558 non-null  float64
 13  edited_at               2859 non-null    object 
 14  content             

In [74]:
data = data.drop("reblog", axis=1)

In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135566 entries, 0 to 135565
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   toot_id                 135566 non-null  object 
 1   created_at              135560 non-null  object 
 2   in_reply_to_id          9427 non-null    object 
 3   in_reply_to_account_id  9427 non-null    object 
 4   sensitive               135560 non-null  object 
 5   spoiler_text            3017 non-null    object 
 6   visibility              135559 non-null  object 
 7   language                120510 non-null  object 
 8   uri                     135559 non-null  object 
 9   url                     135558 non-null  object 
 10  replies_count           135559 non-null  object 
 11  reblogs_count           135558 non-null  float64
 12  favourites_count        135558 non-null  float64
 13  edited_at               2859 non-null    object 
 14  content             

In [76]:
data["account_id"] = ""
data["account_username"] = ""
data["account_acct"] = ""
data["account_display_name"] = ""
data["account_locked"] = False
data["account_bot"] = False

In [77]:
count=0
errors = []
for index, row in data.iterrows():
    try:
        account_str = row.account
        account_str = account_str.replace("\'", "\"").replace('False', 'false').replace("True","true")
        trim_field_index = account_str.find("discoverable")
        account_str = account_str[:trim_field_index-3] + "}"
        account_json = json.loads(account_str)
        data.loc[index, "account_id"] = account_json["id"]
        data.loc[index, "account_username"] = account_json["username"]
        data.loc[index, "account_acct"] = account_json["acct"]
        data.loc[index, "account_display_name"] = account_json["display_name"]
        data.loc[index, "account_locked"] = account_json["locked"]
        data.loc[index, "account_bot"] = account_json["bot"]
    except:
        pass

In [78]:
if 'account_locked' in data:
    data = data.drop("account_locked", axis=1)

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135566 entries, 110344178529920798 to 110348006697669475
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   created_at              135560 non-null  object 
 1   in_reply_to_id          9427 non-null    object 
 2   in_reply_to_account_id  9427 non-null    object 
 3   sensitive               135560 non-null  object 
 4   spoiler_text            3017 non-null    object 
 5   visibility              135559 non-null  object 
 6   language                120510 non-null  object 
 7   uri                     135559 non-null  object 
 8   url                     135558 non-null  object 
 9   replies_count           135559 non-null  object 
 10  reblogs_count           135558 non-null  float64
 11  favourites_count        135558 non-null  float64
 12  edited_at               2859 non-null    object 
 13  content                 135558 non-null  object 
 

In [89]:
data.describe(include="all")

,toot_id,created_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,url,...,emojis,card,poll,application,instance,account_id,account_username,account_acct,account_display_name,account_bot
count,1.355660e+05,135560,9.427000e+03,9427.0,135560,3017,135559,120510,135559,135558,...,135556,48830,360,20713,135556,1.355660e+05,135566,135566,135566,135566
unique,3.634000e+03,2034,2.500000e+02,218.0,5,69,2,33,3547,3546,...,106,1219,10,80,1,2.451000e+03,2431,2451,2400,2
top,1.103442e+17,2023-05-11 03:47:46+00:00,1.103442e+17,1218531.0,False,Spoiler One Piece 1083\n\n#ONEPIECESPOILERS #O...,public,en,https://mastodon.social/users/SauhagPlaneWatch...,https://mastodon.social/@SauhagPlaneWatcher/11...,...,[],{'url': 'https://www.wpc.ncep.noaa.gov/archive...,"{'id': 479146, 'expires_at': datetime.datetime...","{'name': 'Web', 'website': None}",mastodon.social,1.103389e+17,kunev,kunev@blewsky.social,,False
freq,4.100000e+01,598,1.200000e+02,121.0,67110,240,135558,76878,41,41,...,131131,480,40,7058,135556,1.560000e+03,1560,1560,2933,98509
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
mask = data.toot_id == 110344183250998873
data[mask].content

8160    Millennial Media Offensive MMO #71 – Subpoener...
8201    Millennial Media Offensive MMO #71 – Subpoener...
8242    Millennial Media Offensive MMO #71 – Subpoener...
8283    Millennial Media Offensive MMO #71 – Subpoener...
8324    Millennial Media Offensive MMO #71 – Subpoener...
8365    Millennial Media Offensive MMO #71 – Subpoener...
8406    Millennial Media Offensive MMO #71 – Subpoener...
8447    Millennial Media Offensive MMO #71 – Subpoener...
8488    Millennial Media Offensive MMO #71 – Subpoener...
8529    Millennial Media Offensive MMO #71 – Subpoener...
8570    Millennial Media Offensive MMO #71 – Subpoener...
8611    Millennial Media Offensive MMO #71 – Subpoener...
8652    Millennial Media Offensive MMO #71 – Subpoener...
8693    Millennial Media Offensive MMO #71 – Subpoener...
8734    Millennial Media Offensive MMO #71 – Subpoener...
8775    Millennial Media Offensive MMO #71 – Subpoener...
8816    Millennial Media Offensive MMO #71 – Subpoener...
8857    Millen